#GPT-2 Music generation 
2/23

Based off of Train a GPT-2 Text-Generating Model w/ GPU For Free by [Max Woolf](http://minimaxir.com)


In [ ]:
%tensorflow_version 1.x
!pip install -q gpt-2-simple
import gpt_2_simple as gpt2
from datetime import datetime
from google.colab import files
!pip install gdown
!gdown https://drive.google.com/uc?id=[redacted]
!pip install pandas
!pip install msgpack-python
!pip install glob2
!pip install tqdm
!pip install py-midi
!pip install tensorflow
!sudo apt-get install swig
!sudo apt-get install libasound2-dev
!pip install git+https://github.com/vishnubob/python-midi@feature/python3

TensorFlow 1.x selected.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libasound2-dev is already the newest version (1.1.3-5ubuntu0.5).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
  Cloning https://github.com/vishnubob/python-midi (to revision feature/python3) to /tmp/pip-req-build-hhid1rdd
  Running command git clone -q https://github.com/vishnubob/python-

## Downloading GPT-2

If you're retraining a model on new text, you need to download the GPT-2 model first. 

There are three released sizes of GPT-2:

* `124M` (default): the "small" model, 500MB on disk.
* `355M`: the "medium" model, 1.5GB on disk.
* `774M`: the "large" model, cannot currently be finetuned with Colaboratory but can be used to generate text from the pretrained model (see later in Notebook)
* `1558M`: the "extra large", true model. Will not work if a K80/P4 GPU is attached to the notebook. (like `774M`, it cannot be finetuned).

Larger models have more knowledge, but take longer to finetune and longer to generate text. You can specify which base model to use by changing `model_name` in the cells below.

The next cell downloads it from Google Cloud Storage and saves it in the Colaboratory VM at `/models/<model_name>`.

This model isn't permanently saved in the Colaboratory VM; you'll have to redownload it if you want to retrain it at a later time.

In [ ]:
!gdown --id [redacted, contact]
import zipfile
with zipfile.ZipFile("/content/Copy of midis_v1.0.zip","r") as zip_ref:
    zip_ref.extractall("/content/")



Downloading...
From: https://drive.google.com/uc?id=19VENDv6hMSdWagaJ4B0Kut6WRU_mDM9r
To: /content/Copy of midis_v1.0.zip
193MB [00:03, 61.8MB/s]


In [ ]:
import os

path =  "/content/midis"
filenames = os.listdir(path)

for filename in filenames:
  if "mid" in filename:
    os.rename(os.path.join(path , filename), os.path.join(path , filename[-len("Bq0BXYA7riI.mid"):]))


In [ ]:
!rm  -r /content/txt/
!mkdir /content/txt/

In [ ]:
import argparse
import midi
from midi import *
import os
import json
import sys
import json
import random

def gen_vocab(tick_max, include_resolutions = False, resolutions = False):
    events = [i for i in dir(midi.events) if i.find('Event') >= 0]
    vocab = {}
    count = 0

    # tracks
    for i in range(16):
        for e in events:
            vocab[str(i) + e] = count
            count += 1

    for i in range(tick_max + 1):
        vocab[str(i) + 't'] = count
        count += 1

    # channels
    for i in range(128):
        vocab[str(i) + 'c'] = count
        count += 1

    # data array
    for i in range(128):
        vocab[str(i) + 'd'] = count
        count += 1

    # resolutions
    if include_resolutions:
        for r in resolutions:
            vocab[str(r) + 'r'] = count
            count += 1

    with open('vocab.json', 'w') as f:
        f.write(json.dumps(vocab))

    #print("Generated vocab.json in {}".format(os.path.dirname(os.path.realpath('vocab.json'))))

def demidi(midi_dir, data_dir, include_resolutions, tick_max, generate_vocab):
    midifiles = []
    for filename in os.listdir(midi_dir):
        if filename.endswith("mid"):
            # Your code comes here such as 
            #print(filename)
            midifiles.append(filename)

    # 96 is quarter notes, but if giving music with variing notes, do not set this.
    masterresolution = 96
    # mother 3 music is 24 (and highest tick is only 2489)
    # undertale/deltarune is 96

    biggest_tick = 0
    fff = ''
    goodticks = []
    badsongs = []
    resolutions = []
    events_blacklist = ['CopyrightMetaEvent', 'AbstractEvent', 'MetaEvent', 'MarkerEvent', 'SomethingEvent', 'CuePointEvent']
    ds=0
    dl=len(midifiles)
    print(dl)
    for midifile in midifiles:
        
        #print("{}/{}".format(midi_dir, midifile))
        pattern = midi.read_midifile('{}/{}'.format(midi_dir, midifile))


        resolutions.append(pattern.resolution)
        count = 0
        words = []
        word = ""
        write_file = True
        for p in pattern:
            for e in p:
                if e.__class__.__name__ not in events_blacklist:
                    word = ""
                    word += str(count) + e.__class__.__name__.lower()
                    word += str(e.tick) + 't'

                    if tick_max >= 0:
                        if e.tick >= tick_max:
                            badsongs.append(midifile)
                        else:
                            goodticks.append(e.tick)

                        if e.tick > biggest_tick:
                            biggest_tick = e.tick

                    if e.__class__.__name__ not in ['InstrumentNameEvent', 'MarkerEvent', 'TrackNameEvent', 'TimeSignatureEvent', 'SetTempoEvent', 'EndOfTrackEvent', 'CopyrightMetaEvent', 'KeySignatureEvent', 'SmpteOffsetEvent']:
                        word += str(e.channel) + 'c'
                    #else:
                        #print(e)

                    for d in e.data:
                        word += str(d) + 'd'
                    words.append(word)
            count += 1

        if midifile not in badsongs or tick_max < 0:
            with open("{}/{}.txt".format(data_dir, midifile), 'w') as f:
                if include_resolutions:
                    f.write("{} {}".format(pattern.resolution, " ".join(words)))
                else:
                    f.write(" ".join(words))
                f.write("\n")
        resolutions = list(set(resolutions))
        ds+=1
        if ds%100==0:print(ds/dl)
        #print("Largest tick found: {}\nResolutions found: {}".format(biggest_tick, resolutions))

        if generate_vocab:
            gen_vocab(biggest_tick, include_resolutions=include_resolutions, resolutions=resolutions)

demidi("/content/midis","/content/txt",True,-1,False)


10110
0.009891196834817012
0.019782393669634024
0.02967359050445104
0.03956478733926805
0.04945598417408507
0.05934718100890208
0.06923837784371908
0.0791295746785361
0.08902077151335312
0.09891196834817013
0.10880316518298715
0.11869436201780416
0.12858555885262116
0.13847675568743817
0.14836795252225518
0.1582591493570722
0.1681503461918892
0.17804154302670624
0.18793273986152326
0.19782393669634027
0.20771513353115728
0.2176063303659743
0.2274975272007913
0.23738872403560832
0.24727992087042533
0.2571711177052423
0.26706231454005935
0.27695351137487634
0.2868447082096934
0.29673590504451036
0.3066271018793274
0.3165182987141444
0.3264094955489614
0.3363006923837784
0.34619188921859545
0.3560830860534125
0.3659742828882295
0.3758654797230465
0.3857566765578635
0.39564787339268054
0.4055390702274975
0.41543026706231456
0.42532146389713155
0.4352126607319486
0.44510385756676557
0.4549950544015826
0.4648862512363996
0.47477744807121663
0.4846686449060336
0.49455984174085066
0.5044510385

In [ ]:
import re
midifiles = []
for filename in os.listdir("/content/txt"):
    if filename.endswith("txt"): 
        # Your code comes here such as 
        #print(filename)
        midifiles.append(filename)
q = re.compile("1controlchangeevent")
z = re.compile("1noteonevent")
for midifile in midifiles:
    content_new=""
    with open("{}/{}".format("/content/txt", midifile), 'r') as f:
      content = f.read()
      content_new = re.sub("1noteonevent", 'z', content, flags = re.M)
      content_new = re.sub("1controlchangeevent", 'q', content_new, flags = re.M)
    with open("{}/{}".format("/content/txt", midifile), 'w') as f:
      f.write(content_new)


In [ ]:
import fileinput
import sys

def replaceAll(file,searchExp,replaceExp):
    for line in fileinput.input(file, inplace=1):
        if searchExp in line:
            line = line.replace(searchExp,replaceExp)
        sys.stdout.write(line)

replaceAll("/content/midis.txt","1noteonevent","z")
replaceAll("/content/midis.txt","1controlchangeevent","q")


In [ ]:
print()

In [ ]:
#!find "/content/txt/" -type f -print0 | echo "" >> xargs
!find "/content/txt/" -name "*.txt" -exec cat {} + > midis.txt


In [ ]:
!rm -rf /content/txt
!mkdir /content/txt

In [ ]:
cmmt='''import tarfile
import os.path

def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
make_tarfile("midis.tar","/content/txt")'''
#!find "/content/txt/" -type f -print0 | sort -zR | tail -zn +1001 | xargs -0 rm
#print(open('/content/txt/--4b9rCf7fc.mid.txt', "r").read())

384 0settempoevent0t7d161d32d 0timesignatureevent0t4d2d24d8d 0endoftrackevent1t 1controlchangeevent1367t0c64d127d 1controlchangeevent115t0c64d0d 1controlchangeevent246t0c64d127d 1noteonevent341t0c79d56d 1noteonevent11t0c75d51d 1noteonevent258t0c48d40d 1noteonevent328t0c60d47d 1noteonevent271t0c77d69d 1noteonevent1t0c74d56d 1noteonevent249t0c75d0d 1noteonevent30t0c77d0d 1noteonevent2t0c80d71d 1noteonevent10t0c77d58d 1noteonevent265t0c48d0d 1noteonevent5t0c48d48d 1controlchangeevent11t0c64d0d 1controlchangeevent53t0c64d127d 1noteonevent31t0c79d0d 1noteonevent184t0c60d0d 1noteonevent10t0c60d48d 1noteonevent270t0c79d66d 1noteonevent19t0c75d53d 1noteonevent239t0c77d0d 1noteonevent8t0c77d73d 1noteonevent9t0c68d57d 1noteonevent244t0c75d0d 1noteonevent11t0c75d80d 1noteonevent4t0c80d0d 1noteonevent2t0c60d0d 1noteonevent0t0c77d0d 1controlchangeevent2t0c64d0d 1noteonevent1t0c79d0d 1noteonevent3t0c48d0d 1noteonevent4t0c67d58d 1noteonevent5t0c48d53d 1noteonevent6t0c74d0d 1controlchangeevent23t0c64d

In [ ]:
gpt2.download_gpt2(model_name="774M")
sess = gpt2.start_tf_sess()

Fetching checkpoint: 1.05Mit [00:00, 402Mit/s]                                                      
Fetching encoder.json: 1.05Mit [00:00, 5.33Mit/s]
Fetching hparams.json: 1.05Mit [00:00, 274Mit/s]                                                    
Fetching model.ckpt.data-00000-of-00001: 3.10Git [01:16, 40.4Mit/s]                                 
Fetching model.ckpt.index: 1.05Mit [00:00, 382Mit/s]                                                
Fetching model.ckpt.meta: 2.10Mit [00:00, 11.6Mit/s]                                                
Fetching vocab.bpe: 1.05Mit [00:00, 7.10Mit/s]


In [ ]:
import tensorflow as tf

tf.reset_default_graph()

sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, model_name="774M")

Loading pretrained model models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


## Mounting Google Drive

The best way to get input text to-be-trained into the Colaboratory VM, and to get the trained model *out* of Colaboratory, is to route it through Google Drive *first*.

Running this cell (which will only work in Colaboratory) will mount your personal Google Drive in the VM, which later cells can use to get data in/out. (it will ask for an auth code; that auth is not saved anywhere)

In [ ]:
gpt2.mount_gdrive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Uploading a Text File to be Trained to Colaboratory

In the Colaboratory Notebook sidebar on the left of the screen, select *Files*. From there you can upload files:

![alt text](https://i.imgur.com/TGcZT4h.png)

Upload **any smaller text file**  (<10 MB) and update the file name in the cell below, then run the cell.

In [ ]:
file_name = "midis.txt"

If your text file is larger than 10MB, it is recommended to upload that file to Google Drive first, then copy that file from Google Drive to the Colaboratory VM.

## Finetune GPT-2

The next cell will start the actual finetuning of GPT-2. It creates a persistent TensorFlow session which stores the training config, then runs the training for the specified number of `steps`. (to have the finetuning run indefinitely, set `steps = -1`)

The model checkpoints will be saved in `/checkpoint/run1` by default. The checkpoints are saved every 500 steps (can be changed) and when the cell is stopped.

The training might time out after 4ish hours; make sure you end training and save the results so you don't lose them!

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

Other optional-but-helpful parameters for `gpt2.finetune`:


*  **`restore_from`**: Set to `fresh` to start training from the base GPT-2, or set to `latest` to restart training from an existing checkpoint.
* **`sample_every`**: Number of steps to print example output
* **`print_every`**: Number of steps to print training progress.
* **`learning_rate`**:  Learning rate for the training. (default `1e-4`, can lower to `1e-5` if you have <1MB input data)
*  **`run_name`**: subfolder within `checkpoint` to save the model. This is useful if you want to work with multiple models (will also need to specify  `run_name` when loading the model)
* **`overwrite`**: Set to `True` if you want to continue finetuning an existing model (w/ `restore_from='latest'`) without creating duplicate copies. 

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

sess = gpt2.start_tf_sess()

gpt2.finetune(sess,
              dataset="/content/txt",
              model_name='774M',
              steps=-1,
              restore_from='fresh',
              run_name='run1',
              print_every=500,
              sample_every=5000,
              save_every=5000,
              overwrite=True
              )

Loading checkpoint models/774M/model.ckpt
INFO:tensorflow:Restoring parameters from models/774M/model.ckpt


  0%|          | 0/10110 [00:00<?, ?it/s]

Loading dataset...


100%|██████████| 10110/10110 [48:31<00:00,  3.47it/s]


dataset has 701813408 tokens
Training...
[500 | 878.80] loss=1.11 avg=1.11
[1000 | 1741.62] loss=0.89 avg=1.00
[1500 | 2604.57] loss=0.93 avg=0.98
[2000 | 3466.91] loss=0.95 avg=0.97
[2500 | 4328.67] loss=0.93 avg=0.96
[3000 | 5189.44] loss=0.78 avg=0.93
[3500 | 6050.97] loss=0.84 avg=0.92
[4000 | 6913.11] loss=0.85 avg=0.91


After the model is trained, you can copy the checkpoint folder to your own Google Drive.

If you want to download it to your personal computer, it's strongly recommended you copy it there first, then download from Google Drive. The checkpoint folder is copied as a `.rar` compressed file; you can download it and uncompress it locally.

In [ ]:
while True:pass

In [ ]:
gpt2.copy_checkpoint_to_gdrive(run_name='run1')

You're done! Feel free to go to the **Generate Text From The Trained Model** section to generate text based on your retrained model.

## Load a Trained Model Checkpoint

Running the next cell will copy the `.rar` checkpoint file from your Google Drive into the Colaboratory VM.

In [ ]:
gpt2.copy_checkpoint_from_gdrive(run_name='run1')

The next cell will allow you to load the retrained model checkpoint + metadata necessary to generate text.

**IMPORTANT NOTE:** If you want to rerun this cell, **restart the VM first** (Runtime -> Restart Runtime). You will need to rerun imports but not recopy files.

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

sess = gpt2.start_tf_sess()

gpt2.load_gpt2(sess, run_name='run1')

AttributeError: ignored

## Generate Text From The Trained Model

After you've trained the model or loaded a retrained model from checkpoint, you can now generate text. `generate` generates a single text from the loaded model.

In [ ]:
gpt2.generate(sess, run_name='run1')

1 q6t0c64d0d z0t0c75d0d z0t0c72d0d q0t0c64d127d z78t0c69d0d q3t0c64d0d z5t0c73d0d z1t0c67d0d z2t0c67d63d z0t0c76d0d z8t0c71d0d z20t0c75d74d z1t0c68d80d z4t0c49d0d z0t0c69d76d z0t0c72d85d z2t0c72d0d z0t0c67d0d q0t0c64d127d z36t0c67d70d z0t0c73d79d z4t0c77d79d z4t0c72d77d z0t0c69d0d z3t0c67d0d z0t0c73d0d z0t0c80d0d z3t0c74d0d z0t0c66d0d z0t0c68d0d z0t0c71d94d z1t0c74d77d z2t0c67d82d z0t0c80d94d z11t0c69d80d z4t0c68d80d z0t0c71d0d z0t0c69d0d z11t0c69d71d z5t0c64d70d z1t0c74d0d z8t0c69d66d z2t0c72d72d z7t0c73d75d q19t0c64d127d z28t0c67d0d z3t0c64d0d z1t0c69d0d z0t0c67d61d z2t0c69d73d z2t0c72d0d z4t0c72d77d z1t0c69d69d z1t0c70d90d z9t0c69d0d z9t0c69d72d z1t0c69d0d z5t0c69d66d z0t0c69d0d z0t0c64d72d z0t0c69d64d z1t0c69d0d z0t0c69d86d z3t0c69d0d z6t0c69d64d z1t0c69d0d z3t0c69d71d z4t0c69d0d z7t0c69d66d z0t0c69d0d z1t0c69d67d z2t0c69d62d z0t0c69d0d z0t0c71d77d z4t0c69d71d z7t0c69d0d z4t0c69d71d z1t0c69d0d z10t0c69d69d z2t0c69d0d z5t0c69d69d z30t0c69d0d z0t0c69d69d z4t0c69d0d z8t0c69d68d z1t0c6

If you're creating an API based on your model and need to pass the generated text elsewhere, you can do `text = gpt2.generate(sess, return_as_list=True)[0]`

You can also pass in a `prefix` to the generate function to force the text to start with a given character sequence and generate text from there (good if you add an indicator when the text starts).

You can also generate multiple texts at a time by specifing `nsamples`. Unique to GPT-2, you can pass a `batch_size` to generate multiple samples in parallel, giving a massive speedup (in Colaboratory, set a maximum of 20 for `batch_size`).

Other optional-but-helpful parameters for `gpt2.generate` and friends:

*  **`length`**: Number of tokens to generate (default 1023, the maximum)
* **`temperature`**: The higher the temperature, the crazier the text (default 0.7, recommended to keep between 0.7 and 1.0)
* **`top_k`**: Limits the generated guesses to the top *k* guesses (default 0 which disables the behavior; if the generated output is super crazy, you may want to set `top_k=40`)
* **`top_p`**: Nucleus sampling: limits the generated guesses to a cumulative probability. (gets good results on a dataset with `top_p=0.9`)
* **`truncate`**: Truncates the input text until a given sequence, excluding that sequence (e.g. if `truncate='<|endoftext|>'`, the returned text will include everything before the first `<|endoftext|>`). It may be useful to combine this with a smaller `length` if the input texts are short.
*  **`include_prefix`**: If using `truncate` and `include_prefix=False`, the specified `prefix` will not be included in the returned text.

In [ ]:
global graph
graph = tf.get_default_graph() 
global pog
pog=graph

In [ ]:
def getgpt2(refix):
  global pog
  graph = pog
  ree=gpt2.generate(sess,
              length=1024,
              temperature=0.8,
              prefix=refix,
              nsamples=1,
              batch_size=1,
              include_prefix=False
              , return_as_list=True)[0]
  return ree

In [ ]:
st="324 0settempoevent0t7d161d32d 0timesignatureevent0t4d2d24d8d 0endoftrackevent1t "
e=" 1endoftrackevent1t"
g=[]
f="324 "
for i in range(1,10):
  f=getgpt2(f)
  g.append(f[:700])
  f=(f[700:])
g.append(e)
print(''.join(g))

324 63d69d z18t0c76d0d z31t0c64d0d z29t0c69d0d z15t0c63d0d z22t0c47d0d z2t0c64d0d z7t0c53d0d z2t0c72d0d z0t0c65d0d z6t0c65d67d z8t0c69d64d z16t0c55d0d z6t0c53d55d z116t0c49d80d z6t0c54d92d z4t0c69d0d z15t0c69d67d z110t0c64d97d z8t0c69d0d z14t0c69d67d z7t0c53d0d z7t0c73d0d z4t0c73d61d z11t0c69d70d z51t0c66d79d z5t0c69d0d z4t0c69d64d z49t0c78d0d z7t0c78d67d z8t0c69d0d z8t0c69d68d z9t0c67d73d z2t0c72d78d z11t0c69d0d z5t0c69d82d z23t0c60d78d z1t0c74d88d z0t0c69d0d z4t0c69d59d z1t0c73d0d z7t0c73d62d z8t0c69d68d z33t0c73d0d z10t0c73d59d z0t0c69d0d z2t0c69d62d z4t0c67d0d z1t0c69d54d z3t0c78d0d z5t0c68d84d z3t0c69d0d z4t0c69d67d z2t0c74d0d z1t0c68d0d z8t0c69d0d z88t0c55d64d z3t0c69d61d z0t0c53d68d z0t0c65d68d z3t0c69d0d z30t0c69d69d z4t0c55d0d z6t0c54d0d z1t0c69d0d z4t0c51d0d z1t0c69d60d z9t0c69d0d z7t0c69d75d z0t0c65d0d z3t0c69d65d z2t0c71d70d z18t0c65d59d z0t0c69d0d z14t0c69d64d z0t0c66d0d z5t0c66d71d z7t0c62d60d z8t0c69d0d z1t0c69d65d z12t0c69d0d z37t0c49d0d z10t0c49d65d z1t0c66d0d z9t0c66d

For bulk generation, you can generate a large amount of text to a file and sort out the samples locally on your computer. The next cell will generate a generated text file with a unique timestamp.

You can rerun the cells as many times as you want for even more generated texts!

In [ ]:
g=' '.join(g)

In [ ]:
gen_file = 'gpt2_gentext_{:%Y%m%d_%H%M%S}.txt'.format(datetime.utcnow())

gpt2.generate_to_file(sess,
                      destination_path=gen_file,
                      length=500,
                      temperature=0.7,
                      nsamples=100,
                      batch_size=20
                      )

In [ ]:
# may have to run twice to get file to download
files.download(gen_file)

## Generate Text From The Pretrained Model

If you want to generate text from the pretrained model, not a finetuned model, pass `model_name` to `gpt2.load_gpt2()` and `gpt2.generate()`.

This is currently the only way to generate text from the 774M or 1558M models with this notebook.

In [ ]:
g="""96 0settempoevent0t7d161d32d 0timesignatureevent0t4d2d24d8d 0endoftrackevent1t z18t0c76d0d z31t0c64d0d z29t0c69d0d z15t0c63d0d z22t0c47d0d z2t0c64d0d z7t0c53d0d z2t0c72d0d z0t0c65d0d z6t0c65d67d z8t0c69d64d z16t0c55d0d z6t0c53d55d z116t0c49d80d z6t0c54d92d z4t0c69d0d z15t0c69d67d z110t0c64d97d z8t0c69d0d z14t0c69d67d z7t0c53d0d z7t0c73d0d z4t0c73d61d z11t0c69d70d z51t0c66d79d z5t0c69d0d z4t0c69d64d z49t0c78d0d z7t0c78d67d z8t0c69d0d z8t0c69d68d z9t0c67d73d z2t0c72d78d z11t0c69d0d z5t0c69d82d z23t0c60d78d z1t0c74d88d z0t0c69d0d z4t0c69d59d z1t0c73d0d z7t0c73d62d z8t0c69d68d z33t0c73d0d z10t0c73d59d z0t0c69d0d z2t0c69d62d z4t0c67d0d z1t0c69d54d z3t0c78d0d z5t0c68d84d z3t0c69d0d z4t0c69d67d z2t0c74d0d z1t0c68d0d z8t0c69d0d z88t0c55d64d z3t0c69d61d z0t0c53d68d z0t0c65d68d z3t0c69d0d z30t0c69d69d z4t0c55d0d z6t0c54d0d z1t0c69d0d z4t0c51d0d z1t0c69d60d z9t0c69d0d z7t0c69d75d z0t0c65d0d z3t0c69d65d z2t0c71d70d z18t0c65d59d z0t0c69d0d z14t0c69d64d z0t0c66d0d z5t0c66d71d z7t0c62d60d z8t0c69d0d z1t0c69d65d z12t0c69d0d z37t0c49d0d z10t0c49d65d z1t0c66d0d z9t0c66d56d z12t0c69d61d z15t0c66d0d z69t0c69d0d z4t0c69d64d z8t0c66d0d z4t0c66d58d z5t0c69d0d z7t0c69d70d z3t0c65d0d z5t0c65d62d z5t0c69d0d z71t0c69d0d z6t0c69d56d z3t0c66d0d z10t0c66d57d z7t0c69d0d z7t0c69d63d z4t0c65d0d z174t0c61d84d z1t0c69d0d z10t0c69d85d z5t0c65d0d z2t0c65d65d z1t0c69d0d z0t0c69d67d z0t0c69d0d z1t0c65d0d z8t0c69d72d z4t0c69d67d z2t0c59d0d z0t0c72d0d z1t0c69d0d z5t0c69d77d z7t0c61d0d z9t0c61d73d z1t0c69d0d z16t0c66d0d z183t0c69d63d z1t0c66d67d z6t0c69d0d z12t0c69d74d z5t0c65d61d z7t0c69d0d z1t0c69d63d q0t0c64d127d z2t0c68d67d z1t0c69d0d z14t0c69d72d z0t0c69d0d z8t0c69d74d z8t0c65d0d z11t0c66d0d z32t0c69d0d z5t0c59d76d z2t0c69d73d z1t0c65d62d z0t0c69d0d z5t0c69d75d z3t0c69d72d z2t0c65d0d z5t0c65d60d z0t0c69d0d z8t0c69d77d z1t0c59d0d z1t0c69d74d z4t0c65d0d z2t0c69d0d z0t0c69d75d z4t0c69d0d z4t0c69d60d z3t0c65d66d z0t0c69d0d z86t0c69d0d z11t0c69d66d z0t0c66d72d z9t0c69d0d z8t0c66d0d z35t0c69d69d z3t0c65d0d z0t0c69d0d z0t0c69d71d z0t0c62d0d z0t0c69d57d z5t0c65d64d q30t0c64d0d z2t0c69d0d z1t0c69d70d z0t0c62d69d z3t0c69d60d z1t0c75d70d z2t0c69d0d z0t0c65d0d z0t0c69d0d z0t0c69d94d z3t0c70d89d z0t0c69d96d z1t0c63d72d z6t0c69d0d q4t0c64d127d z1t0c68d0d z2t0c69d0d z3t0c69d65d z0t0c69d0d z0t0c70d0d z6t0c70d66d z2t0c69d73d z4t0c70d0d z11t0c69d67d q46t0c64d0d z1t0c69d0d z0t0c69d89d z3t0c67d79d z4t0c69d0d z0t0c70d0d z5t0c63d0d z0t0c62d0d z1t0c55d88d z0t0c74d80d z1t0c69d75d z2t0c70d80d z9t0c69d0d z1t0c65d76d z1t0c69d70d z2t0c65d0d z0t0c69d0d z5t0c69d79d z1t0c69d0d z0t0c69d81d z0t0c66d82d z3t0c69d78d z3t0c62d72d q22t0c64d127d z49t0c69d0d z8t0c69d78d z1t0c69d0d z5t0c69d83d z2t0c69d0d z1t0c69d70d z0t0c62d0d z49t0c69d74d z2t0c66d0d z31t0c69d0d z1t0c69d75d z1t0c62d75d z0t0c69d0d z1t0c69d74d z1t0c62d0d z0t0c69d0d z10t0c69d72d z0t0c69d0d z31t0c70d0d z17t0c69d84d z0t0c69d76d z2t0c62d69d z2t0c69d0d z7t0c69d84d z2t0c62d0d z3t0c66d82d z9t0c69d0d z16t0c66d0d z32t0c70d84d z0t0c69d90d z0t0c62d92d z1t0c69d95d z0t0c67d0d z3t0c62d0d z11t0c69d0d z3t0c69d80d z0t0c70d0d z13t0c69d84d z0t0c67d77d z0t0c69d0d z1t0c69d89d z1t0c62d78d z1t0c69d0d z4t0c69d77d z0t0c67d0d z27t0c69d0d z0t0c62d0d z0t0c67d81d z1t0c69d81d z3t0c62d70d z1t0c69d0d z6t0c69d61d q31t0c64d0d z8t0c69d0d z0t0c69d83d z3t0c66d82d z7t0c66d0d z1t0c69d80d z1t0c62d0d z0t0c69d0d z0t0c69d93d z3t0c62d75d z0t0c69d83d z2t0c62d0d z1t0c67d0d z7t0c69d0d z0t0c69d91d z0t0c67d83d z5t0c62d75d z3t0c69d78d z2t0c59d85d z3t0c60d82d z30t0c69d0d z3t0c69d84d z0t0c67d0d z2t0c69d81d z0t0c62d0d z2t0c69d0d z3t0c67d84d z1t0c69d88d z7t0c62d74d z0t0c69d95d z0t0c63d83d z2t0c59d0d z1t0c69d85d z0t0c67d0d z3t0c69d0d z0t0c67d91d z0t0c59d90d z2t0c66d90d z0t0c70d94d z0t0c70d0d z4t0c68d76d z0t0c69d95d z0t0c67d89d z3t0c69d0d z4t0c69d74d z1t0c62d0d z0t0c69d0d z0t0c69d74d z0t0c62d87d z0t0c67d0d z0t0c69d0d z8t0c69d87d z0t0c67d88d z6t0c69d0d z4t0c69d63d z0t0c67d0d z7t0c67d80d z0t0c64d76d z2t0c66d0d z6t0c69d82d z0t0c62d0d z2t0c62d69d z3t0c69d0d z9t0c69d80d z3t0c69d0d z0t0c69d87d z0t0c67d0d z6t0c67d75d z0t0c69d95d z6t0c67d0d z8t0c69d0d z11t0c69d84d z0t0c72d80d z0t0c69d88d z1t0c62d0d z7t0c69d85d z5t0c63d0d z2t0c62d74d z0t0c69d0d z0t0c68d0d z9t0c69d88d z0t0c63d72d z0t0c69d0d z5t0c69d77d z0t0c69d0d z11t0c69d74d z0t0c67d85d z0t0c69d80d z5t0c67d0d z3t0c69d0d z8t0c68d80d z0t0c69d86d z0t0c67d69d z0t0c59d0d z0t0c69d87d z4t0c69d0d z10t0c69d77d z0t0c67d0d z3t0c69d69d z1t0c67d56d z0t0c69d0d z1t0c69d74d z1t0c63d0d z4t0c69d0d z0t0c69d91d z0t0c63d82d z0t0c69d77d z1t0c69d0d z8t0c69d90d z20t0c69d0d z5t0c69d78d z1t0c69d0d z7t0c69d67d z1t0c59d69d z0t0c69d68d z1t0c63d0d z31t0c69d77d z8t0c69d0d z7t0c69d77d z5t0c69d0d z2t0c69d59d z1t0c66d80d z0t0c62d78d z0t0c69d0d z0t0c66d0d z1t0c69d73d z3t0c66d78d z0t0c69d0d z2t0c69d66d z0t0c69d69d z0t0c61d78d z2t0c69d0d z0t0c66d0d z0t0c62d0d z0t0c69d0d z3t0c69d80d z1t0c66d79d z1t0c69d84d z0t0c59d0d z0t0c69d83d z0t0c63d81d z3t0c69d0d z1t0c67d85d z1t0c62d78d z0t0c69d80d z3t0c63d0d z0t0c66d0d z0t0c69d0d z0t0c69d72d z1t0c66d68d z0t0c69d81d z0t0c62d0d z0t0c61d0d z0t0c69d0d z0t0c69d75d z4t0c61d74d z0t0c69d81d z12t0c69d0d z6t0c69d76d z0t0c69d0d z4t0c69d80d z4t0c69d0d z3t0c66d0d z0t0c62d75d z0t0c66d78d z1t0c69d65d z0t0c59d68d z0t0c69d80d z1t0c63d81d z2t0c69d0d z1t0c62d0d z0t0c74d79d z0t0c69d80d z2t0c66d0d z0t0c66d75d z1t0c69d86d z0t0c63d0d z1t0c59d0d z0t0c69d0d z0t0c69d55d z0t0c39d74d z0t0c76d68d z0t0c71d75d z0t0c72d46d z0t0c69d69d z1t0c69d0d z1t0c69d70d z0t0c63d74d z0t0c69d0d z0t0c66d0d z0t0c69d87d z2t0c62d92d z0t0c66d95d z0t0c69d76d z0t0c59d71d z0t0c69d0d z0t0c62d0d z0t0c66d0d z1t0c69d77d z1t0c62d91d z0t0c66d87d z0t0c69d69d z0t0c63d0d z0t0c69d0d z0t0c66d0d z0t0c69d101d z2t0c62d0d z1t0c66d78d z0t0c69d78d z0t0c63d74d z0t0c69d0d z0t0c69d71d z2t0c59d0d z7t0c69d0d z8t0c70d88d z0t0c69d92d z3t0c66d0d z0t0c64d57d z0t0c69d92d z3t0c62d82d z0t0c69d0d z4t0c62d0d z0t0c69d68d z2t0c69d0d z4t0c69d80d z2t0c62d90d z0t0c69d85d z0t0c66d84d z0t0c69d78d z1t0c67d83d z0t0c59d70d z2t0c69d0d z0t0c69d80d z1t0c66d0d z9t0c69d0d z3t0c66d85d z0t0c69d86d z1t0c69d79d z0t0c62d0d z0t0c69d0d z0t0c62d74d z1t0c69d79d z1t0c66d0d z0t0c69d80d z1t0c69d0d z0t0c70d0d z7t0c69d83d z1t0c71d0d z0t0c69d0d z0t0c66d0d z1t0c69d88d z1t0c70d87d z4t0c66d83d z1t0c69d83d z4t0c71d73d z0t0c67d0d z0t0c69d0d z2t0c67d80d z0t0c69d74d z0t0c69d76d z0t0c62d0d z0t0c64d0d z0t0c69d0d z0t0c66d0d z0t0c69d82d z0t0c66d93d z0t0c69d94d z0t0c68d0d z0t0c69d0d z2t0c69d77d 1endoftrackevent1t
"""
import re
g = re.sub('z', '1noteonevent', g)
g = re.sub('q','1controlchangeevent',g)


text_file = open("file1.txt", "w")
n = text_file.write(g)
text_file.close()

In [ ]:
print(g)

96 0settempoevent0t7d161d32d 0timesignatureevent0t4d2d24d8d 0endoftrackevent1t 1noteonevent2t0c67d65d 1noteonevent5t0c55d46d 1noteonevent18t0c50d0d 1noteonevent11t0c50d26d 1noteonevent68t0c67d0d 1noteonevent0t0c79d0d 1noteonevent8t0c79d68d 1noteonevent3t0c67d66d 1noteonevent9t0c55d0d 1noteonevent7t0c55d41d 1noteonevent107t0c67d0d 1noteonevent0t0c79d0d 1noteonevent4t0c67d60d 1noteonevent5t0c79d60d 1noteonevent4t0c55d0d 1noteonevent7t0c55d49d 1noteonevent15t0c50d0d 1noteonevent10t0c50d32d 1noteonevent90t0c55d0d 1noteonevent0t0c79d0d 1noteonevent7t0c79d68d 1noteonevent1t0c55d53d 1noteonevent102t0c50d0d 1noteonevent0t0c55d0d 1noteonevent0t0c79d0d 1noteonevent6t0c79d68d 1noteonevent0t0c50d40d 1noteonevent3t0c55d50d 1noteonevent110t0c50d0d 1noteonevent0t0c55d0d 1noteonevent0t0c79d0d 1noteonevent5t0c79d68d 1noteonevent1t0c55d52d 1noteonevent3t0c50d44d 1noteonevent107t0c50d0d 1noteonevent0t0c55d0d 1noteonevent0t0c79d0d 1noteonevent4t0c55d52d 1noteonevent2t0c79d69d 1noteonevent1t0c50d42d 1noteo

In [ ]:
!git clone https://github.com/stephwag/midi-text.git

fatal: destination path 'midi-text' already exists and is not an empty directory.


# Etcetera

If the notebook has errors (e.g. GPU Sync Fail), force-kill the Colaboratory virtual machine and restart it with the command below:

In [ ]:
!kill -9 -1

In [ ]:
!python3 /content/midi-text/remidi.py --datafile "/content/file1.txt" --outfile "/content/file1.mid"


settempoevent
timesignatureevent
endoftrackevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
noteonevent
not

In [ ]:
!pip install midi2audio
!sudo apt-get install fluidsynth




Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libfluidsynth1 libqt5x11extras5 qsynth
Suggested packages:
  fluid-soundfont-gs timidity jackd
The following NEW packages will be installed:
  fluid-soundfont-gm fluidsynth libfluidsynth1 libqt5x11extras5 qsynth
0 upgraded, 5 newly installed, 0 to remove and 10 not upgraded.
Need to get 120 MB of archives.
After this operation, 150 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluid-soundfont-gm all 3.1-5.1 [119 MB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfluidsynth1 amd64 1.1.9-1 [137 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 fluidsynth amd64 1.1.9-1 [20.7 kB]
Get:4 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libqt5x11extras5 amd64 5.9.5-0ubuntu1 [8,596 B]
Get:5 http://archive.ubuntu.com/ubuntu bionic/uni

In [ ]:
from midi2audio import FluidSynth
fs=FluidSynth()
fs.midi_to_audio('file1.mid', 'output.wav')

In [ ]:
!sudo apt-get install -y fluid-soundfont-gm
!dpkg -L fluidsynth


Reading package lists... Done
Building dependency tree       
Reading state information... Done
fluid-soundfont-gm is already the newest version (3.1-5.1).
0 upgraded, 0 newly installed, 0 to remove and 31 not upgraded.
/.
/usr
/usr/bin
/usr/bin/fluidsynth
/usr/share
/usr/share/doc
/usr/share/doc/fluidsynth
/usr/share/doc/fluidsynth/NEWS.gz
/usr/share/doc/fluidsynth/README.md.gz
/usr/share/doc/fluidsynth/TODO
/usr/share/doc/fluidsynth/copyright
/usr/share/man
/usr/share/man/man1
/usr/share/man/man1/fluidsynth.1.gz
/usr/share/doc/fluidsynth/changelog.Debian.gz


# Reflection(4/3):
While I did get this to work (poorly), I had little knowledge of how GPT-2 actually worked when I made this(The BPE in particular), which was probably the reason why it didn't work very well. This project was a good learning experience on how to process and prepare data.